# Introduction to constrained multi-objective management optimization (under uncertainty)

### yeah, its getting deep!

In the two PESTPP-OPT notebooks ([freyberg opt 1](../part2_08_opt/freyberg_opt_1.ipynb) and [freyberg opt 2](../part2_08_opt/freyberg_opt_2.ipynb)), we introduced the concept of constrained management optimization under uncertainty.  We saw standard risk-neutral optimization and then piled on the learning and concepts with the idea of chances, chance constraints, risk/reliability, and stacks.  So if you are reading this notebook, we assume that you are familiar with these concepts and terminology by now. If not, we recommend you go through the PESTPP-OPT notebooks first.

Ok, so now let's talk about the nature of constraints and _multiple_ objective functions.  In the Freyberg example, we have been treating the sw-gw exchange flux and the aggregate groundwater extraction rate for each stress period as a "hard" inequality constraint (thou shall not violate!). However, in real-world decision-making settings, although there may be a general stakeholder preference to avoid unwanted outcomes,  the exact nature of that avoidance is often not known. "Sure we want to keep some groundwater flowing into the surface-water system but we also want have plenty of water to drink". As with many things in the messy real-world, this is all very imprecise and mucky...so how can we, science nerds, deal with this messiness?

Well, one way is to use so-called "multi-objective" optimization. Here the goal is to map the trade-off between competing objectives. We move away from giving the "unique" optimal solution (that respects our constraints and risk-stance). Instead, we provide the range of solutions (that respect the constraints), and their corresponding effects on the competing objectives. 

Unfortunately, this kind of trade-off mapping is very (very (_very!_)) computationally expensive because, in most cases, we have to resort to "global" evolutionary-type algorithms.  Note that "multi-objective" doesn't mean go crazy with objectives.  Five or six is probably the most that be used for algorithmic reasons.  

> ### WARNING! 
Running this notebook can take a looong time. You may wish to quickly read through, (__IMPORTANT!__) set the number of PESTPP-MOU worker agents adequate for your machine, and then let MOU do its business for a few hours (or even over-night). Alternatively, you can read through the notebook on the repository GitHub page without needing to run anything. That being said, going through the pain of having to listen to your computer groan as it runs the model a gazillion times is a powerful lesson into the benefits of simple model design. (_Recall that the Freyberg model takes seconds to run; imagine doing this with a model that takes 1 minute or...dare I say it...several minutes! #somuchpain_)

PESTPP-MOU and some of the concepts are also introduced in __[White et al (2022)](https://www.sciencedirect.com/science/article/pii/S1364815222000226)__. If you are interested in learning more about multi-objective optimization, #[LMGTFY](https://www.urbandictionary.com/define.php?term=lmgtfy): "pareto frontier", "pareto dominance", "nsga-II", etc. 


### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see ["freyberg intro to model"](../part0_02_intro_to_freyberg_model/intro_freyberg_model.ipynb) notebook), and the high-dimensional PEST dataset prepared in the ["freyberg pstfrom pest setup"](../part2_01_pstfrom_pest_setup/freyberg_pstfrom_pest_setup.ipynb)" and ["observation and weights"](../part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb) notebooks.

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 15}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd

To maintain continuity in the series of tutorials, we will use the PEST-dataset prepared in the ["freyberg OPT"](../part2_08_opt/freyberg_opt_1.ipynb) tutorial. Run the next cell to copy the necessary files across. Note that you will need to run the previous notebooks in the correct order beforehand.

Specify the path to the PEST dataset template folder. Recall that we will prepare our PEST dataset files in this folder, keeping them separate from the original model files. Then copy across pre-prepared model and PEST files:

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("..","part2_08_opt","freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_08_opt/freyberg_opt_1.ipynb' notebook")

shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')

### Inspect the PEST Dataset

OK. We can now get started.

Load the PEST control file as a `Pst` object. We are going to use the PEST control file that was created in the ["freyberg OPT"](../part2_08_opt/freyberg_opt_1.ipynb) tutorial.

In [ ]:
pst = pyemu.Pst(pst_path)

### PESTPP-MOU

`PESTPP-MOU` implements a constrained multiple and single objective "global" optimization using evolutionary algorithms. In the previous OPT notebook, we introduced some terminology. Now we need to introduce some more! (These are all described in in __[White et al (2022)](https://www.sciencedirect.com/science/article/pii/S1364815222000226)__)


 - "individual": an optimization problem candidate solution. So just a decision variable vector - one value for each decision variable. Or a single set of decision variable values (think "realisation" of decision variable parameters);
 - "population":  well, a collection of individuals (think "ensemble" of decision variable parameters);
 - "generation": a complete cycle of the evolutionary algorithm (think "iteration"), which involves generating a new, "child" population by combining parents, evaluating the children's fitness (running the population through the model), and the (natural) selection, where the "best" individuals from the parent and child population are kept.  ("best" is where things get complicated...we will get to that later);
 - "generator":  the algorithmic process to generate a child population.  Differential evolution is the default in PESTPP-MOU but there are others
 - "selector": the algorithmic process to pick the "best" individuals in the population to move to the next generation.  For single objective formulations, this is trivial. For multiobjective formulations, selection is also complex.


Well, there you have it - you are now ready for PESTPP-MOU! but wait, how big should the population be?  How many generations should I use?  Great questions!  Generally it is said that the population should be about twice as large as the number of decision variables.  As for generations, lots (and this is the problem that makes this computationally expensive!).  Like 50, 100, or more generations are not uncommon...

Now for some important aspects of using PESTPP-MOU in (heavily) constrained formulations.  


#### Initial Decision Variable Population
As we will see below, it is important to make sure at least a few individuals in the initial population are (nearly) feasible.  This helps greatly, by providing some _individuals_ with a recognizable evolutionary advantage.  

For our Freyberg case, we know that extracting less groundwater will result in the sw-gw exchange constraints being satisfied. (We saw this in the previous OPT notebooks...also it is logical: less pumping means more water available to discharge to the stream). 

So, let's generate an initial population ourselves.  We can use pyEMU to sample decision variable values (e.g. pumping rates), assuming lower upper bounds (e.g. forcing the initial decision variable values to be lower). This increases the likelihood that individuals will result in a feasible solution (e.g. that respects the _constraints_).

Conceptually, although we want to explore using larger pumping rates, we want to do that from within the feasible region so that PESTPP-MOU can maintain a (largely) feasible population.

OK, here goes some hackery to get the initial decision variable population:

In [ ]:
par = pst.parameter_data
# set all parameters fixed
par.loc[:,"partrans"] = "fixed"
# get the decision variable parameter data
dvpar = par.loc[par.pargp=="decvars",:]
# free only the decision variables
par.loc[dvpar.parnme,"partrans"] = "none"
# temporariliy lower the decision variable upper bounds
par.loc[dvpar.parnme,"parubnd"] = 1.0

## generate initial population from smaller decision variable space
# first sepcify how may individuals in the population; 
# these should generally be twice the number of decision variables
num_reals = 160
# sample 160 sets of decision variables from a unform distribution
dvpop = pyemu.ParameterEnsemble.from_uniform_draw(pst,num_reals=num_reals)
# record to external file for PESTPP-MOU
dvpop.to_csv(os.path.join(t_d,"initial_dvpop.csv"))
# tell PESTPP-MOU about the new file
pst.pestpp_options["mou_dv_population_file"] = 'initial_dvpop.csv'
#revert the changes to the parameter data 
par.loc[:,"partrans"] = "none"
# reset the decision variable upper bound
par.loc[dvpar.parnme,"parubnd"] = 6.0

#### Specify Objectives
Now we need to tell PESTPP-MOU what objectives it needs to deal with.  Let's try to (1) maximize the cumulative volume of groundwater extracted while also trying to (2) maximize the volume of groundwater discharged to surface water.  

From the [user manual](https://github.com/usgs/pestpp/blob/develop/documentation/pestpp_users_manual.md#1325-running-pestpp-mou): 
> Constraints/objectives are identified in exactly the same way as PESTPP-OPT: via the observation/prior information equation group names. Group names that start with “less_than” are identified as less-than constraints/minimization objectives; group names that start with “greater_than” are identified as greater-than constraints/maximization objectives. The distinction between constraints and objectives is made via the mou_objectives argument. Any observation and prior information equation names (not group names) passed via MOU_OBJECTIVES are treated as objectives, not constraints. While it may seem tempting to make all constraints objectives, in practice, the algorithm elements encoded in PESTPP-MOU can tolerate up to 5 objectives, and, more realistically, 2-3 objectives may be a better choice.

So, like with PESTPP-OPT, with PESTPP-MOU we use the `obgnme` naming trick  to tell it the direction to seek for each objective. Adding the prefix "less_than" (or "greater_than") to the observation group name, informs PESTPP-MOU that it should minimize (or maximize)  observation values in this group. 

In [ ]:
# pass the observation NAMES to the "mou_objectives" variable
pst.pestpp_options["mou_objectives"] = ["oname:cum_otype:lst_usecol:sfr_totim:4383.5",
                                        "oname:cum_otype:lst_usecol:wel_totim:4383.5"]

# remove the PESTPP-OPT prior info based objective function
pst.prior_information = pst.null_prior
obs = pst.observation_data
# turn off the groundwater use constraints - this will let PESTPP-MOU explore more options for us
obs.loc[obs.apply(lambda x: x.weight > 0 and "wel" in x.obsnme,axis=1),"weight"] = 0.0

# now define the direction of each objective and give them a non-zero weight:
obs.loc[pst.pestpp_options["mou_objectives"],'weight'] = 1.0
obs.loc[pst.pestpp_options["mou_objectives"],'obgnme'] = "less_than_obj"                

#### Population Size

As mentioned, generally the population size should be at least double the number of decision variables. Recall that we have already generated a initial decision variable population with 160 individuals - we just need to tell PESTPP-MOU to stick to that number

In [ ]:
print("number of dec vars:",dvpar.shape[0])

In [ ]:
# some additional PESTPP-MOU options:
pst.pestpp_options["mou_population_size"] = 160 #twice the number of decision variables
pst.pestpp_options["mou_save_population_every"] = 1 # save lots of files! 
                                                    # but this way we can inspect how MOU progressed    

#### Test run

In [ ]:
pst.control_data.noptmax = 0 #just for a test run
pst.write(pst_path,version=2)  
pyemu.os_utils.run("pestpp-mou freyberg_mf6.pst",cwd=t_d)

Ok, assuming that cell completed without errors, if you look at the screen output or the rec file, you can verify that the optimization problem is formulated as desired.

## Run PESTPP-MOU

Some testing on this problem indicates the problem stabilizes on a pareto frontier around 30-40 generations; let's run it for 50. We specify the number of generations with good'ole NOPTMAX. So set that to 50. Yeah, you heard that right - 50. Welcome to MOU:

In [ ]:
pst.control_data.noptmax = 50
pst.write(pst_path,version=2)

# Attention!

You must specify the number which is adequate for ***your*** machine! Make sure to assign an appropriate value for the following `num_workers` variable:

In [ ]:
num_workers = 15 # update according to your available resources!

Then specify the folder in which the PEST manager will run and record outcomes. It should be different from the `t_d` folder. 

In [ ]:
m_d = os.path.join('master_mou_1')
if os.path.exists(m_d):
    shutil.rmtree(m_d)

The following cell deploys the PEST agents and manager and then starts the run using `pestpp-mou`. Run it by pressing `shift+enter`.

If you wish to see the outputs in real-time, switch over to the terminal window (the one which you used to launch the `jupyter notebook`). There you should see `pestpp-mou`'s progress. 

If you open the tutorial folder, you should also see a bunch of new folders there named `worker_0`, `worker_1`, etc. These are the agent folders. `pyemu` will remove them when PEST finishes running.

This run should take a while to complete (depending on the number of workers and the speed of your machine). If you get an error, make sure that your firewall or antivirus software is not blocking `pestpp-mou` from communicating with the agents (this is a common problem!).

#### This will take a quite a while. So feel free to start it and go to bed! haha

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-mou","freyberg_mf6.pst",num_workers=num_workers,worker_root=".",
                           master_dir=m_d)

### Processing PESTPP-MOU

Ok, lets see what's in the master dir:

In [ ]:
[f for f in os.listdir(m_d) if f.startswith("freyberg_mf6")]

Holy crap that's a lot of files!  A brief description of these files:

 - "freyberg_mf6.XX.dv_pop.csv": the decision variable population at the end of generation XX
 - "freyberg_mf6.XX.obs_pop.csv": the corresponding observation values for the population at the end of generation XX
 - "freyberg_mf6.XX.archive.dv_pop.csv": the archive population at the end of generation XX; the archive only contains "quality" individuals
 - "freyberg_mf6.XX.archive.obs_pop.csv": the corresponding observation values for the archive population at the end generation XX
 - "freyberg_mf6.pareto.summary.csv": a summary of the pareto dominance and feasibility (think "fitness") of each individual across all generations
 - "freyberg_mf6.pareto.acrhive.summary.csv": a summary of the pareto dominance and feasibility (think "fitness") of each individual in the archive across all generations.
 
   
Let's inspect that archive summary file:

In [ ]:
df = pd.read_csv(os.path.join(m_d,"freyberg_mf6.pareto.archive.summary.csv"))
df.tail()

We only want to see feasible and non-dominated individuals, so let's filter on those criteria.

The "nsga2_front" is a measure of domination. Individuals in the first front (e.g. ==1) are nondominated. These are the ones we want to keep. The next cell selects only feasible and nondominated individuals.

In [ ]:
feas_front_df = df.loc[df.apply(lambda x: x.nsga2_front==1 and x.is_feasible==1,axis=1),:]

More hackery alert!  Here is just a plotting function to help us visualize the trade-off between our two objectives:

In [ ]:
def plot_pareto(gen):
    """make a cross plot for a given generation.  Note the axes ranges are 
    sync'd across all generations so you can compare the plots more easily
    
    """
    df = feas_front_df.loc[feas_front_df.generation==gen,:]
    fig,axes = plt.subplots(2,2,figsize=(10,10))
    objs = pst.pestpp_options["mou_objectives"]
    onames = [o.split("usecol:")[1].split('_')[0] for o in objs]
    axes[0,0].hist(df.loc[:,objs[0]],alpha=0.5,facecolor="0.5")
    axes[0,0].set_title(onames[0])
    axes[1,1].set_title(onames[1])
    axes[0,0].set_xlim(feas_front_df.loc[:,objs[0]].min(),
                      feas_front_df.loc[:,objs[0]].max())
    axes[1,1].set_xlim(feas_front_df.loc[:,objs[1]].min(),
                      feas_front_df.loc[:,objs[1]].max())

    axes[1,1].hist(df.loc[:,objs[1]],alpha=0.5,facecolor="0.5")
    axes[1,0].scatter(df.loc[:,objs[0]],df.loc[:,objs[1]],marker=".",c="0.5")
    axes[0,1].scatter(df.loc[:,objs[1]],df.loc[:,objs[0]],marker=".",c="0.5")
    axes[1,0].set_title("{0} vs {1}".format(onames[0],onames[1]))
    axes[0,1].set_title("{0} vs {1}".format(onames[1],onames[0]))
    axes[1,0].set_xlim(feas_front_df.loc[:,objs[0]].min(),
                      feas_front_df.loc[:,objs[0]].max())
    axes[1,0].set_ylim(feas_front_df.loc[:,objs[1]].min(),
                      feas_front_df.loc[:,objs[1]].max())
    axes[0,1].set_ylim(feas_front_df.loc[:,objs[0]].min(),
                      feas_front_df.loc[:,objs[0]].max())
    axes[0,1].set_xlim(feas_front_df.loc[:,objs[1]].min(),
                      feas_front_df.loc[:,objs[1]].max())
    for ax in [axes[0,0],axes[1,1]]:
        ax.set_yticks([])
        ax.set_xlabel("$L^3$")
    for ax in [axes[1,0],axes[0,1]]:
        ax.set_xlabel("$L^3$")
        ax.set_ylabel("$L^3$")
        


    plt.tight_layout()
    return fig,axes

First let's check out the final generation - our best results:

In [ ]:
fig,axes = plot_pareto(feas_front_df.generation.max())
plt.show();

So what is that?  In the diagonal we see the histograms of both objectives - the distribution of their values in the final population.  The off-diagonals are where things get interesting:  These show the pareto frontier between extracting more groundwater and leaving groundwater for surface-water/groundwater exchange.  (Take care to note the negative values on the axes).

Remember __this is still a constrained problem__, meaning that each of these individuals still meets the required surface-water/groundwater exchange flux constraint for each stress period. Even within the feasible space, there is considerable variation in the groundwater extraction rate, meaning there are choices to be made (The shape of the pareto frontier is a classic.). This plot enables decision-makers to visualize the trade-off between more/less pumping and more/less sw-gw discharge.

While we are at it, let's see the initial generation results:

In [ ]:
fig,axes = plot_pareto(feas_front_df.generation.min())
plt.show()

yikes!  What we are seeing is the result of our "front-loading" where we sampled over an initial decision variable range that we knew would give us better opportunities for feasibility. Effectively, we started in a relatively "small" area of decision variable space,  which we knew was likely to be feasible. Then, PESTPP-MOU went off and explored a larger area.

This is a critical aspect of PESTPP-MOU usage.  Just naively throwing a population at PESTPP-MOU in a heavily constrained problem will probably lead to no feasible individuals #badtimes. But if PESTPP-MOU is given an initial population with at least a few feasible individuals, it will strongly prefer those feasible candidates in the algorithmic generation process so that the entire population will move toward feasibility.

Now, you may have noted a distinct lack of "parameter uncertainty" being mentioned in this notebook - worry not, we bring the conversation back there in the [next PESTPP-MOU tutorial](../part2_09_mou/freyberg_mou_2.ipynb). Stick around for the next episode! 